In [47]:
from dotenv import load_dotenv
from affine import Affine
import os
import numpy as np
import pathlib
from pathlib import PurePosixPath
from pyproj import Transformer
import s3fs
import zarr

# load bucket credentials as environment variables
dotenv_dir = os.environ.get("CREDENTIAL_DOTENV_DIR", os.getcwd())
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [61]:
import s3fs

bucket = os.environ.get("OSC_S3_BUCKET_DEV")
s3 = s3fs.S3FileSystem(
        key=os.environ.get("OSC_S3_ACCESS_KEY_DEV", None),
        secret=os.environ.get("OSC_S3_SECRET_KEY_DEV", None),
    )

group_path = str(PurePosixPath(bucket, "hazard/hazard.zarr"))
scenario = "historical"
year = "1985"
full_path = str(PurePosixPath(bucket, f"hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_{scenario}_{year}"))


The Zarr group is located at path `physrisk-hazard-indicators-dev01/hazard/hazard.zarr`.
The particular array is located at path `physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985`.

In [62]:
print(f"Contents of 'driectory' {full_path}:")
contents = s3.ls(full_path)
print(f"{len(contents)} files")
print(contents)


Contents of 'driectory' physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985:
765 files
['physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985/.zarray', 'physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985/.zattrs', 'physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985/0.0.21', 'physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985/0.0.22', 'physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985/0.0.23', 'physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985/0.0.24', 'physrisk-hazard-indicators-dev01/hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unpr

Note that this comprises files .zarray, .zattrs, and the (non-empty) chunks which have filenames in the format {chunk_z, chunk_y, chunk_x}, e.g. 0.1.21

In [63]:
with s3.open(full_path + "/.zarray") as f:
    print(f.read().decode())

{
    "chunks": [
        5,
        1000,
        1000
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<f4",
    "fill_value": "NaN",
    "filters": null,
    "order": "C",
    "shape": [
        5,
        39420,
        38374
    ],
    "zarr_format": 2
}


The array is (5 returns) x (39420 spatial y coordinates) x (38374 spatial x coordinates). The array is split into chunks where each chunk is 5 x 1000 x 1000.

In [64]:
with s3.open(full_path + "/.zattrs") as f:
    print(f.read().decode())

{
    "crs": "+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs=True",
    "dimensions": [
        "index",
        "latitude",
        "longitude"
    ],
    "index_name": "return period (years)",
    "index_values": [
        10,
        30,
        100,
        300,
        1000
    ],
    "transform_mat3x3": [
        100.0,
        0.0,
        2648099.999999999,
        0.0,
        -99.99999999999999,
        5404499.999999997,
        0.0,
        0.0,
        1.0
    ]
}


This defines the dimensions, gives the CRS for the spatial dimensions (x/longitude and y/latitude), defines the index values for the z dimension (which contains each return period).

Transform_mat3x3 provides the affine transform needed to find pixels from latitude and longitude.

In [65]:
# For accessing 

store = s3fs.S3Map(root=group_path, s3=s3, check=False)
root = zarr.group(store=store)
z = root["inundation/river_tudelft/v2/flood_depth_unprot_historical_1985"]


In [66]:
t = z.attrs["transform_mat3x3"]
crs = z.attrs["crs"]
transform = Affine(t[0], t[1], t[2], t[3], t[4], t[5])
lat, lon = (51.7426, -1.2669)
transproj = Transformer.from_crs("epsg:4326", crs, always_xy=True)
x, y = transproj.transform([lon], [lat])

px, py = ~transform * (x[0], y[0])
print(f"pixel values x/y for longitude/latitude are {px}/{py}")

pixel values x/y for longitude/latitude are 8982.571352041778/21631.33479974286


In [67]:
print(z[:, int(py), int(px)])

[2.2507873 2.4166772 2.5981872 2.7464173 2.8810973]


In [68]:
# some code to do the same thing but fast for a large number of x and y coordinates

coords = np.vstack((x, y, np.ones(len(x))))  # type: ignore
inv_trans = ~transform
mat = np.array(inv_trans).reshape(3, 3)
frac_image_coords = mat @ coords
print(frac_image_coords)
index_values = z.attrs.get("index_values", [0])
image_coords = np.floor(frac_image_coords).astype(int)
iy = np.repeat(image_coords[1, :], len(index_values))
ix = np.repeat(image_coords[0, :], len(index_values))
iz = np.tile(np.arange(z.shape[0]), image_coords.shape[1]) 
data = z.get_coordinate_selection((iz, iy, ix)) 
print(data)

[[8.98257135e+03]
 [2.16313348e+04]
 [1.00000000e+00]]
[2.2507873 2.4166772 2.5981872 2.7464173 2.8810973]


In [69]:
# doing the same thing locally

test_output_dir = '/Users/joemoorhouse/Code/hazard/tests/test_output/' # this contains a copy of hazard/hazard.zarr/inundation/river_tudelft/v2/flood_depth_unprot_historical_1985
local_store = zarr.DirectoryStore(os.path.join(test_output_dir, 'hazard', 'hazard.zarr'))
root_local = zarr.group(store=local_store)
z_local = root_local["inundation/river_tudelft/v2/flood_depth_unprot_historical_1985"]

In [70]:
print(z_local[:, int(py), int(px)])

[2.2507873 2.4166772 2.5981872 2.7464173 2.8810973]


A note on performance. When we request a set of coordinates from Zarr, Zarr works out which chunks it needs to obtain, decompress and which pixels it needs to look up from those chunks. The only non-trivial parts in terms of performance are:
1) getting the chunk into memory
2) decompressing the chunk

For an S3 store, 1 takes most of the time and it is important that the machine doing the decompression and look-up has a fast connection to the bucket (as I write, I am running this on a laptop with a broadband connection in a different continent to the bucket, an example of a slow connection!).

S3 is generally 'fast enough' in that requesting order 100,000 coordinates takes order ~10s (depending on clustering of lats/lons) but of course we can get orders of magnitude faster than this, e.g. storing on a solid state hard disk, or distributed memory containing uncompressed tiles.

Moreover, Zarr provides a number of stores (we have used only the S3 and local so far) to manage the chunks.
https://zarr.readthedocs.io/en/stable/api/storage.html
